In [1]:
%matplotlib inline

In [2]:
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import random
import pickle
from collections import Counter
from nltk.stem import WordNetLemmatizer
import torch
import torch.nn as nn
import torchvision

lemmatizer = WordNetLemmatizer()
hm_lines = 1000000


def read_lexicon():
    global lexicon 

    lexi = open("lexicon.pickle" , 'rb')
    lexicon = pickle.load(lexi)
    
    return lexicon
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
ip_size , op_size = 431 ,  2

class NN(nn.Module):
    def __init__(self , ip_size , op_size):
        super().__init__()
        
        self.fc1 = nn.Linear(ip_size , 1000)
        self.fc2 = nn.Linear(1000 , 3000)
        self.fc3 = nn.Linear(3000 , 300)
        self.fc4 = nn.Linear(300 ,op_size)
        self.relu = nn.ReLU()
        
    def forward(self , x):
        x = self.fc1(x)
        x = self.fc2(self.relu(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        
        return x

In [4]:
model_loaded = torch.load("NNmodel.pt")

In [5]:
def user_test(sample , lexicon):
    
    c_w = word_tokenize(sample.lower())
    c_w = [lemmatizer.lemmatize(i) for i in c_w]
    samples = np.zeros(len(lexicon))
    for  word in c_w:
        if word.lower() in lexicon:
            index_value = lexicon.index(word.lower())
            samples[index_value] += 1
            
    samples = list(samples)  
    output = []
    output.append(samples)
    output = np.array(output)
    return output

def predict_data(data ):
    lexicon = read_lexicon()
    data  = user_test(data , lexicon)
    data = torch.from_numpy(data)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    data = data.to(device)
    data = data.float()
    predict = model_loaded(data)
#     print(predict[0])
#     p = predict.cpu()
#     p  = p.detach().numpy()
#     p = str(p[0])
#     p = p.split()
#     p.remove(']')
# #     p = p.
#     print(p)
    prediction = torch.max(predict , 1) [1]
    result = prediction.cpu()
    result  = result.numpy()
#     if int(result) == 0:
#         print("--------------------------")
#         print( "Positive")
#         print("--------------------------")
#     elif int(result) == 1:
#         print("--------------------------")
#         print("Negative")
#         print("--------------------------")
#     else:
#         print(int(result) , "Got this value")
        
    return int(result)
    

In [6]:
# data = input()
# result =  predict_data(data)


# Web Scrapping

In [7]:
 # Get link of the review page

In [8]:
import matplotlib.pyplot as plt
import numpy as np
from bokeh.models import ColumnDataSource ,HoverTool
from bokeh.plotting import figure
from bokeh.io import show  ,output_notebook
output_notebook()
def plot(total , pos , neg , title):
    
    categories = ['Positive' , 'Negative']
    color = ['Green' , 'Red']
    top = [pos , neg]
    source = ColumnDataSource(data = dict(categories=categories , top=top , color = color , legend=['Positive','Negative']))
    p = figure(x_range=categories ,plot_width=600 , plot_height=600 , title = title, tools='hover',)
    p.vbar(x='categories' ,top= 'top'  ,width=0.5 ,color='color' ,legend='legend' , source=source, alpha=0.5)
    hover = p.select(dict(type=HoverTool))
    p.xaxis.axis_label = "Categories"
    p.yaxis.axis_label = "Count"
    p.axis.major_tick_line_color  ='green'
    p.axis.major_label_text_color = "blue"
    p.grid.grid_line_dash='solid'

    p.grid.grid_line_alpha = 0.5
    hover.tooltips = [('Bar Type','@categories') , ('No of Review','@top')]
    show(p)

Loading BokehJS ...

In [20]:
# link = 
from string import punctuation
import requests
from bs4 import BeautifulSoup
from wordcloud import WordCloud, STOPWORDS
title = None
def pl_wordcloud(ls):
    st = 'Bukku No Commends'
    for l in ls:
        st +=' '
        st+=str(l)
    wordcloud = WordCloud(background_color='black' , 
                         stopwords = STOPWORDS ,
                         min_font_size=10 ).generate(st)
    
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()
def give_review(link):
    
    result  = requests.get(link) 
    soup = BeautifulSoup(result.content)
    global title
    try :
        title = soup.find_all('div',{'class':['col-12-12']})[4]
        title = BeautifulSoup(str(title))
        title= title.find('span')
        title = title.text
    except Exception as NoneType:
        title = "Some Flipkart Product"
    print(title)
    
    link_replaced = link.replace("/p/" , "/product-reviews/")
    # 5 review pages 
    val=8
    links_app = []
    for i in range(val):
        links_app.append(f'{link_replaced}&page={i+1}')


    data = ''
    data_got  = ''
    for i , link in enumerate(links_app):
        try :        
            result  = requests.get(link) 
            data = str(data + ' ' +str(result.content))
            soup = BeautifulSoup(result.content)
            mydivs = soup.find_all('div' , {"class" : ["col-12-12"]})
            data_got = f"{data_got} {mydivs} "
        except Exception as e:
            print(f"Error at link: {i+1} ::: {e}")

    fi = open('mydivs.html' , 'w')
    fi.write(data)
    fi.close()


#     img = BeautifulSoup(img)
    soup = BeautifulSoup(data_got)
    

    soup = BeautifulSoup(str(soup))
    table = soup.find_all('div' ,{"class":"row"})

    print("----------------")

    ps = list(soup.find_all('p'))
    ps.pop(0)


    p_review = []
    i = 0
    for _ in range(len(ps)):
    #     print("i = " ,i )
        try:
            p_review.append(ps[i])        
        except Exception as e:
            pass
        i +=4



    # fi = open('mydivs.html' , 'w')
    # fi.write(str(p_review))
    # fi.close()

    soup = BeautifulSoup(str(p_review))
    pt = []
    for p in soup.find_all('p'):
        pt.append(p.text)


    for i , p in enumerate(pt):
        if p in punctuation:
            pt.pop(i)
    
#     pl_wordcloud(pt)
    positive = negative = neutral = 0
    total = len(pt)
    for p in pt:
        result = predict_data(p)
        if result == 0:
            positive += 1
        elif result == 1:
            negative +=1
        else:
            neutral +=1
    
    print(f"Total Review Analaysed = {len(pt)} \n Positive Review = {positive} \n Negative Review = {negative}")
    plot(total = total , pos = positive ,neg= negative , title = title)
    
    qases(pt)
    


In [25]:
import validators

from ipywidgets import interact, widgets
from IPython.core.display import display, HTML

from IPython.display import display
from IPython.display import clear_output

from IPython.display import HTML, display

display(HTML("""<h1>Buticku The AI</h1>
             <h3>A Combination of <i>Bukku</i> and <i>Tikku</i></h3>
             <h3>Flipkart Assistant</h3>
             <br>
             <br>
             <h4> Paste the product link you want to get review</h4>
            """ ))

def qases(commands):
    coms = str('')
    for i in commands:
        coms = coms +" "+i
    
    from firebase import firebase
    firebaseref = firebase.FirebaseApplication('https://buticku16.firebaseio.com/')
    display(HTML("""<h1>Buticku The AI</h1>
             <h3>Flipkart Assistant</h3>
             <br>
             <br>
             <h4> Ask Your Questions  , If No Questions type exit and enter</h4>
            """ ))
    exit = False
    while not exit:
        question = str(input("Ask Your Question?")).lower()
        if question == 'exit':
            exit = True
        else:
            exit_text = 'empty_and_exit_now'
            firebaseref.put('/','answer',exit_text)
            firebaseref.put('/','text',coms)
            firebaseref.put('/','question',question)
            
            while 1:
                answer = firebaseref.get('/answer',None)
                if answer != exit_text and len(answer) >=1:
                    display(HTML(f"""<h1 style="color:rgba(100,10,50,70)">Question :{question}</h1>  <h2 style="color:rgba(10,50,100.60)"><i>Answer:{answer}</i></h2>"""))

                    firebaseref.put('/','answer',exit_text)
                    break
            

def validate(link):
    if not validators.url(link):
        print("Link Not valid")
    else:
        start = time.time()
        print("Processing..")
        give_review(link)
        end = time.time()
        print(f"Total Time Taken: {int(end -start)} Seconds")
        
#         display(HTML(img))

        


import time

def on_button_clicked(b):    

    clear_output()
#     val = box.value
    val = b

    validate(val)

# box = widgets.Text(description="paste link here")
# display(box)

# button = widgets.Button(description="Ask Bukku!")
# display(button)
# button.on_click(on_button_clicked)

on_button_clicked(input("Paste the Link of Product"))

        


Processing..
Asus ZenFone Max Pro M2 (Blue, 32 GB)  (3 GB RAM)
----------------
Total Review Analaysed = 160 
 Positive Review = 126 
 Negative Review = 34


Ask Your Question? is this worthy product?


Ask Your Question? tell me more about camera?


Ask Your Question? what about camera


Ask Your Question? thank you


Ask Your Question? exit


Total Time Taken: 901 Seconds


In [11]:
    display(HTML("""<h1>Buticku The AI</h1>
             <h3>A Combination of <i>Bukku</i> and <i>Tikku</i></h3>
             <br>
             <br>
             <h4> Ask Your Questions  , If No Questions type exit and enter</h4>
            """ ))

# Testing Area

In [63]:
# categories = ['Positive' , 'Negative']
# color = ['Green' , 'Red']
# top = [50 , 70]
# source = ColumnDataSource(data = dict(categories=categories , top=top , color = color , legend=['Positive','Negative']))
# p = figure(x_range=categories ,plot_width=600 , plot_height=600 , title = title, tools='hover',)
# p.vbar(x='categories' ,top= 'top'  ,width=0.5 ,color='color' ,legend='legend' , source=source, alpha=0.5)
# hover = p.select(dict(type=HoverTool))
# p.xaxis.axis_label = "Categories"
# p.yaxis.axis_label = "Count"
# p.axis.major_tick_line_color  ='green'
# p.axis.major_label_text_color = "blue"
# p.grid.grid_line_dash='solid'

# p.grid.grid_line_alpha = 0.5
# hover.tooltips = [('Bar Type','@categories') , ('No of Review','@top')]
# show(p)

In [19]:
# def check_review(link):
    
#     result  = requests.get(link) 
#     soup = BeautifulSoup(result.content)
#     global title
#     try :
#         title = soup.find_all('div',{'class':['col-12-12']})[4]
#         print(title)
#         title = BeautifulSoup(str(title))
#         title= title.find('span')
#         title = title.text
#     except Exception as NoneType:
#         title = "Some Flipkart Product"
#     print(title)
    
# check_review('https://www.flipkart.com/asus-zenfone-max-pro-m2-blue-32-gb/p/itmfb22gjfzty4am?pid=MOBFB22GBSBNYM9Y&srno=s_1_2&otracker=search&otracker1=search&lid=LSTMOBFB22GBSBNYM9YM5ZCNR&fm=SEARCH&iid=adb57bbd-6a3c-46d1-b49e-9306515d773e.MOBFB22GBSBNYM9Y.SEARCH&ppt=SearchPage&ppn=Search&ssid=eqmu9eovbnbwndhc1559299048524&qH=cd47795a46574b9b')

In [ ]:
# def callui():
    
# def noui():
#     v=input("Enter Link Here in Jupyter lab")
#     start = time.time()  
#     validate(v)
#     end = time.time()
#     print(f"Total Time Taken: {int(end -start)} Seconds")
    
# def labornote(v):
#     if v ==0:
#         callui()
#     else:
#         noui()
        
# labornote(input("Enter '0' if you'r not in jupyter lab"))
